In [28]:
import numpy as np
import pandas as pd
import datetime as dt
cols = ['JRNY_TYPE_ID','ORIG','DEST','SALE_STR_DATE','TRAVEL_STR_DATE','SEASON_STR_DATE','MIN_STAY_VAL',
        'MAX_STAY_VAL','AP_VALUE','RBD']
for i in range(1,8):
    cols.append('DAY_'+str(i))
path = 'test_10000_scaled.csv'
df = pd.read_csv(path, usecols=cols, keep_default_na=False)

In [29]:
df.head()

,JRNY_TYPE_ID,ORIG,DEST,SALE_STR_DATE,TRAVEL_STR_DATE,SEASON_STR_DATE,MIN_STAY_VAL,MAX_STAY_VAL,AP_VALUE,RBD,DAY_1,DAY_2,DAY_3,DAY_4,DAY_5,DAY_6,DAY_7
0,2,15,36,59,59,228,7.000000,262800,4449.313287,3,1,1,1,1,0,0,1
1,2,15,78,144,144,195,6.533127,525600,4449.313287,7,0,0,0,1,1,1,1
2,2,38,2,59,59,242,6.533127,525600,4449.313287,7,1,1,1,1,0,0,1
3,2,37,62,59,59,186,7.000000,262800,4449.313287,0,1,1,1,1,0,0,1
4,2,42,52,59,59,187,7.000000,262800,4449.313287,0,1,1,1,1,0,0,1


In [ ]:
#default values to 0

In [31]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [20]:
df.head()

,JRNY_TYPE_ID,ORIG,DEST,SALE_STR_DATE,TRAVEL_STR_DATE,SEASON_STR_DATE,MIN_STAY_VAL,MAX_STAY_VAL,AP_VALUE,RBD,DAY_1,DAY_2,DAY_3,DAY_4,DAY_5,DAY_6,DAY_7
0,2,15,36,-0.728922,59,228,7,262800,,3,1,1,1,1,0,0,1
1,2,15,78,0.430411,144,195,,525600,,7,0,0,0,1,1,1,1
2,2,38,2,-0.728922,59,242,,525600,,7,1,1,1,1,0,0,1
3,2,37,62,-0.728922,59,186,7,262800,,0,1,1,1,1,0,0,1
4,2,42,52,-0.728922,59,187,7,262800,,0,1,1,1,1,0,0,1


In [ ]:
#check for NaN and empty